<a href="https://colab.research.google.com/github/brandonmoss124/mgmt467-analytics-portfolio/blob/main/Lab1_AI_Assisted_SQL_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1 – AI-Assisted SQL (Annotated)
**Week 3 (Thu) – Prompting & SQL Foundations**

**Goal:** Use Gemini as a co‑pilot to write and understand basic SQL against the *Superstore* dataset in BigQuery.

**Deliverable:** Complete this notebook (prompts, code, outputs), push to GitHub, submit link to Brightspace.

## 0) Setup & Connection

In [ ]:
#@title Install libraries
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow --upgrade

from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os

print("Libraries imported.")

In [ ]:
#@title Authenticate & set project (EDIT PROJECT_ID)
auth.authenticate_user()
print("✅ Authenticated.")

PROJECT_ID = "YOUR_PROJECT_ID_HERE"  # <-- EDIT THIS
assert PROJECT_ID != "YOUR_PROJECT_ID_HERE", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

In [ ]:
#@title Quick connection test (optional)
DATASET = "superstore_data"
TABLE   = "sales"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

try:
    preview = bq.query(f"SELECT * FROM {BQ_TABLE} LIMIT 5").to_dataframe()
    display(preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠ Could not query the table. Double-check dataset/table names.")
    print(e)

---
## 1) Prompting Approach
- **“What”** → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** → `COUNT`, `GROUP BY`
- **“Who is best”** → `SUM`, `ORDER BY`, `LIMIT`

### 1A) “What” — Unique Sub-Categories in the West
**Business Q:** List all **unique** `Sub_Category` where `Region = 'West'`.

**Gemini prompt (copy into Gemini):

In [ ]:
prompt = """
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
"""
print(prompt)

In [ ]:
# Reference SQL (you can paste Gemini's instead)
sql_what = f"""
SELECT DISTINCT Sub_Category
FROM `{PROJECT_ID}.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category;
"""
df_what = bq.query(sql_what).to_dataframe()
display(df_what)

**Line‑by‑line explanation (example):**
- `SELECT DISTINCT Sub_Category` → return each sub‑category once.
- `FROM <project>.superstore_data.sales` → which table to query.
- `WHERE Region = 'West'` → filter to West.
- `ORDER BY Sub_Category` → sort alphabetically.

### 1B) “How many” — Orders by Ship Mode
**Business Q:** How many orders per `Ship_Mode`?

**Gemini prompt:**

In [ ]:
prompt = """
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
"""
print(prompt)

In [ ]:
# Reference SQL
sql_how_many = f"""
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `{PROJECT_ID}.superstore_data.sales`
GROUP BY Ship_Mode
ORDER BY order_count DESC;
"""
df_how = bq.query(sql_how_many).to_dataframe()
display(df_how)

### 1C) “Who is best” — Top 5 Most Profitable Customers
**Business Q:** Identify top 5 customers by total profit.

**Gemini prompt:**

In [ ]:
prompt = """
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
"""
print(prompt)

In [ ]:
# Reference SQL
sql_top5 = f"""
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `{PROJECT_ID}.superstore_data.sales`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 5;
"""
df_top5 = bq.query(sql_top5).to_dataframe()
display(df_top5)

---
## 2) Challenge – Author Your Own Prompts

### Challenge 1 – Average discount for Technology (East)
**Your precise Gemini prompt (example):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Compute the average 'Discount' where Category = 'Technology' and Region = 'East'.
# OUTPUT: `avg_discount`.
```

In [ ]:
sql_c1 = f"""
SELECT AVG(Discount) AS avg_discount
FROM `{PROJECT_ID}.superstore_data.sales`
WHERE Category = 'Technology' AND Region = 'East';
"""
df_c1 = bq.query(sql_c1).to_dataframe()
display(df_c1)

### Challenge 2 – Unique customers per Segment
**Your precise Gemini prompt (example):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: For each 'Segment', count distinct 'Customer_ID' as `unique_customers`, sorted desc.
```

In [ ]:
sql_c2 = f"""
SELECT Segment, COUNT(DISTINCT Customer_ID) AS unique_customers
FROM `{PROJECT_ID}.superstore_data.sales`
GROUP BY Segment
ORDER BY unique_customers DESC;
"""
df_c2 = bq.query(sql_c2).to_dataframe()
display(df_c2)

---
## 3) Reflection – DIVE
- **Discover** / **Investigate** / **Validate** / **Extend**